<a href="https://colab.research.google.com/github/jelkinsjames/17D_code_examples/blob/main/242L4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## EE 242 Lab 4 – Digital Filtering

Authors: Julian James, Dylan Imayama, Jason Bently, Neil Lindgren

(This should be a markup cell, which means that when you run it you just get formated text.)

In [1]:
from scipy.io import wavfile as wav
import numpy as np
import matplotlib.pyplot as plt
!pip install simpleaudio
import simpleaudio as sa
import decimal
import math
from scipy import signal as sig
%matplotlib inline

ModuleNotFoundError: ignored

## Summary

In this lab, we will consider different types of digital filters (specifically discrete-time, linear, 
time-invariant filters) and look at their characterization in both time and frequency.

## Lab 4 turn in checklist

•	Pre-lab (upload to canvas before lab)

•	Lab 4 Jupyter notebook with code for the first 4 exercises assignment in separate cells. Each assignment cell should contain markdown cells (same as lab overview cells) for the responses to lab report questions. Include your lab members’ names at the top of the notebook.

•	1 individual Jupyter notebook with code + markdown cells for the last exercise

Note: The pre-lab should be done individually, and all other assignments should be completed in groups of 2-3 people.


## Assignment 1 -- Different Filter Implementations


In [ ]:
# Part A

   # plot_phase and fs are 0 by default, feel free to change the default values
    # If plot_phase is 0, do not plot the phase response, else plot the phase response
    # If fs is 0, the x axis would be in radians, otherwise it would be in Hz based on the sampling frequency provided
    # Use signal.freqz to get your frequency response 

def plot_mag_freq_response(b, a, plot_phase, fs):
    w, hf = sig.freqz(b,a)
    if (fs != 0):
        w = w*fs/(2*np.pi)
    if (plot_phase == 0):
        plt.plot(w, 20*np.log10(np.abs(hf)))
        plt.ylim(-100,10)

    else:  
        plt.figure(figsize = (25,10))
        plt.subplot(1,2,1)
        plt.plot(w, 20*np.log10(np.abs(hf)))
        plt.ylim(-100,10)
      
        plt.subplot(1,2,2)
        plt.plot(w, np.rad2deg(np.unwrap(np.angle(hf))))
     

In [ ]:
# Part B
# Hint: Use signal.lfilter 
def plot_impulse_response(b, a, imp_length):
    signal = np.zeros(imp_length)
    signal[0] = 1
    y = sig.lfilter(b, a, signal)
    plt.stem(y)

In [ ]:
# Part C
# Use signal.firwin and signal.butter
fir_b = sig.firwin(20, 0.15)
iir_b, iir_a = sig.butter(10, 0.15, 'lowpass')

In [ ]:
#IIR lowpass magnitude and frequency
plot_mag_freq_response(iir_b, iir_a,1,0)

In [ ]:
#IIR impulse
plot_impulse_response(iir_b, iir_a, 100)

In [ ]:
#FIR lowpass magnitude and phase
plot_mag_freq_response(fir_b,1,1,0)

In [ ]:
#FIR impulse
plot_impulse_response(fir_b, 1, 100)

###  Discussion
The two filters produce absoluetely different results. The IIR filter generates a tradiational phase and magnitude plot and a sinusoidal response. The FIR filter, on the other hand, generates a non-tradiational magnitude and phase, and a triangular impulse response. The FIR filter is beneficial for situations where you would like a finiate response with sharp edges. The IIR filter may be better for sound processing where you do not want sharp dips, or spikes.

## Assignment 2 -- Different Filter Implementations for Smoothing Signals

In [ ]:
# Part A
# Base and noise signal from lab 2
time  = np.arange(0,2,1/1000) # associated time vector that corresponds to 2 seconds
n     = len(time)
p     = 10 # points for piecewise linear signal
amp = 20   # amplitude range of base signal
k = 20
base = np.interp(np.linspace(0,p,n),np.arange(0,p),np.random.rand(p)*amp)

# create some random noise to be added to the abve base signals
noiseamp = 2
noise  = noiseamp * np.random.randn(n)
noisy_signal = np.add(base, noise)

fig = plt.figure(figsize = (10,5))

plt.plot(noisy_signal, linewidth = 1, color = 'blue', label="Noisy Signal") 
plt.plot(base, linewidth = 1, color = 'red', label ="Base")
plt.legend(loc="upper right")
plt.ylim([0,25])  
plt.xticks(fontsize=10)
plt.yticks(fontsize=10) 

plt.xlabel('time', fontsize = 15)
plt.ylabel('amplitude', fontsize = 15)
plt.title('Noisey vs Base')

In [ ]:
# Part B
# Filtered signal using a convolution function
hfilt = np.ones(k+1) * 1/(k+1)
filtsig1 = np.convolve(noisy_signal, hfilt)

# Filtered signal using a lfilter function
signal = np.zeros(n)
signal[0] = 1
fir_b = sig.firwin(10, 0.15)
filtsig2 = sig.lfilter(hfilt,signal, noisy_signal)

fig = plt.figure(figsize = (10,3))
plt.plot(filtsig2, linewidth = 1, color = 'blue')
plt.plot(filtsig1, linewidth = 1, color = 'red')

plt.xticks(fontsize=10)
plt.yticks(fontsize=10) 

plt.xlabel('time', fontsize = 15)
plt.ylabel('amplitude', fontsize = 15)
plt.title('convolution(red) vs. lfilter(blue)')

In [ ]:
# Part C
plot_mag_freq_response(hfilt, signal,1,0)

###  Discussion

A moving window average filter is a type of butterworth filter, or low pass filter, that does not have control over its bandwidth for a fixed amount of taps. The moving window average filter will have set coefficients while the butterworth filter is able to set different values set for each tap, hence controlling the frequency selectivity of the filter.

## Assignment 3 -- Filtering an Audio Signal

In [ ]:
# Part A
# Recall the use of numpy.fft.fft from the previous lab


# Horn fast fourier transform
fs, horn_arr = wav.read('horn11short.wav')
segment = horn_arr[0:1024]
xhf = np.fft.fft(segment)


# Creating filters
b, a = sig.butter(8, 550/(fs/2), 'highpass')
w, hf = sig.freqz(b, a)
signal = np.zeros(100)
signal[0] = 1
ht = sig.lfilter(b, a, signal)
filtered_horn = sig.lfilter(b, a, horn_arr)
filtered_horn_segment = sig.lfilter(b, a, segment)


# Plotting
fig = plt.figure(figsize = (15, 8))

plt.subplot(2,2,1)
plt.plot(w, 20 * np.log10(np.abs(hf)))
plt.title('Magnitude Response of Filter')

plt.subplot(2,2,2)
plt.plot(ht)
plt.title('Impulse Response of Filter')

plt.subplot(2,2,3)
plt.plot(np.linspace(0, fs/2, 512), np.abs(xhf[0:512]))
plt.title('Log Magnitude for Original Horn FFT')

plt.subplot(2,2,4)
plt.plot(np.linspace(0, fs/2, 512), np.abs(np.fft.fft(filtered_horn_segment)[0:512]))
plt.title('Log Magnitude for Filtered Horn FFT')


# Playing unfiltered and filtered audios
wav_horn = sa.WaveObject.from_wave_file('horn11short.wav')
play_music = wav_horn.play()
play_music.wait_done()

wav.write('filtered_horn', fs, filtered_horn.astype('int16'))
wav_hornfft = sa.WaveObject.from_wave_file('filtered_horn')
play_music_a = wav_hornfft.play()
play_music_a.wait_done()

In [ ]:
# Part B


# Creating noise
noisy_horn = np.add(horn_arr, 1000*np.random.randn(len(horn_arr)))
noisy_horn_segment = np.add(segment, 1000*np.random.randn(len(segment)))

# Plotting
fig = plt.figure(figsize = (15, 4))

plt.subplot(1,2,1)
plt.plot(np.linspace(0, fs/2, 512), np.abs(xhf[0:512]))
plt.title('Log Magnitude for Original Horn Frequency Content')

plt.subplot(1,2,2)
plt.plot(np.linspace(0, fs/2, 512), np.abs(np.fft.fft(noisy_horn_segment)[0:512]))
plt.title('Log Magnitude for Noisy Horn Frequency Content')

# Playing original and noisy audios
wav_horn = sa.WaveObject.from_wave_file('horn11short.wav')
play_music = wav_horn.play()
play_music.wait_done()

wav.write('noisy_horn', fs, noisy_horn.astype('int16'))
wav_hornfft = sa.WaveObject.from_wave_file('noisy_horn')
play_music_a = wav_hornfft.play()
play_music_a.wait_done()

In [ ]:
# Part C


# Creating filters
b, a = sig.butter(8, 1800/(fs/2), 'lowpass')
w, hf = sig.freqz(b, a)
signal = np.zeros(100)
signal[0] = 1
ht = sig.lfilter(b, a, signal)
filtered_noisy_horn = sig.lfilter(b, a, noisy_horn)
filtered_noisy_horn_segment = sig.lfilter(b, a, noisy_horn_segment)


# Plotting
fig = plt.figure(figsize = (15, 8))

plt.subplot(2,2,1)
plt.plot(w, 20 * np.log10(np.abs(hf)))
plt.title('Magnitude Response of Filter')

plt.subplot(2,2,2)
plt.plot(ht)
plt.title('Impulse Response of Filter')

plt.subplot(2,2,3)
plt.plot(np.linspace(0, fs/2, 512), np.abs(np.fft.fft(noisy_horn_segment)[0:512]))
plt.title('Log Magnitude for Original Noisy Horn Frequency Content')

plt.subplot(2,2,4)
plt.plot(np.linspace(0, fs/2, 512), np.abs(np.fft.fft(filtered_noisy_horn_segment)[0:512]))
plt.title('Log Magnitude for Filtered Noisy Horn Frequency Content')


# Playing original, noisy, and filtered noisy audios
wav_horn = sa.WaveObject.from_wave_file('horn11short.wav')
play_music = wav_horn.play()
play_music.wait_done()

wav_hornfft = sa.WaveObject.from_wave_file('noisy_horn')
play_music_a = wav_hornfft.play()
play_music_a.wait_done()

wav.write('filtered_horn_2', fs, filtered_noisy_horn.astype('int16'))
wav_hornfft = sa.WaveObject.from_wave_file('filtered_horn_2')
play_music_a = wav_hornfft.play()
play_music_a.wait_done()

###  Discussion

While the first harmonic is attenuated and the magnitude of the spike at the first harmonic in the frequency response plot is reduced by a few orders of magnitude, it is still the most prominent harmonic out of all the harmonics in the audio file. The volume is decreased because it was attenuated so much, but the first harmonic remains the strongest one. The impulse response of the HPF has a sharper initial spike than the impulse response of the LPF. After the initial spike in each impulse response, the LPF impulse response has tighter waveforms. The differences in these impulse responses are caused by differences in the filters themselves, as evident in the magnitude response plots. The HPF has a sharp cutoff, and attenuates low frequencies very quickly, whereas the LPF magnitude response is much softer, indicating that higher frequencies are attenuated less sharply.

## Assignment 4 -- Implementing a 3-Band Audio Equalizer

In [ ]:
# Part A

b_lp, a_lp = sig.butter(5, 1/4, 'lowpass')
b_bp, a_bp = sig.butter(5, (1/4, 1/2), 'bandpass')
b_hp, a_hp = sig.butter(5, 1/2, 'highpass')

w_lp, hf_lp = sig.freqz(b_lp, a_lp)
w_bp, hf_bp = sig.freqz(b_bp, a_bp)
w_hp, hf_hp = sig.freqz(b_hp, a_hp)

plt.plot(w_lp, 20 * np.log10(np.abs(hf_lp)))
plt.plot(w_bp, 20 * np.log10(np.abs(hf_bp)))
plt.plot(w_hp, 20 * np.log10(np.abs(hf_hp)))
plt.title('Magnitude Responses of Butterworth LPF, BPF, and HPF')

In [ ]:
# Part B

def audio_equalizer(audio, G1, G2, G3):
    b_lp, a_lp = sig.butter(5, 1/4, 'lowpass')
    b_bp, a_bp = sig.butter(5, (1/4, 1/2), 'bandpass')
    b_hp, a_hp = sig.butter(5, 1/2, 'highpass')
    
    y_lp = sig.lfilter(b_lp, a_lp, audio)
    y_bp = sig.lfilter(b_bp, a_bp, audio)
    y_hp = sig.lfilter(b_hp, a_hp, audio)
    
    y_lp = y_lp * (10 ** (1/20 * G1))
    y_bp = y_bp * (10 ** (1/20 * G2))
    y_hp = y_hp * (10 ** (1/20 * G3))
    return (y_lp + y_bp + y_hp)

In [ ]:
# Part C
fs, music_arr = wav.read('music.wav')
combine_chan   = np.add(music_arr[:, 0]/2 , music_arr[:, 1]/2) 

wav_music = sa.WaveObject.from_wave_file('music.wav')
play_music = wav_music.play()
play_music.wait_done()

a = audio_equalizer(music_arr[:, 0], 0, 0 ,0)
b = audio_equalizer(music_arr[:, 1], 0, 0 ,0)
combine_chan  = np.vstack([a/2 ,b/2]) 
chan = combine_chan.T

wav.write('music_a', fs, chan.astype('int16'))
wav_music_a = sa.WaveObject.from_wave_file('music_a')
play_music_a = wav_music_a.play()
play_music_a.wait_done()

In [ ]:
# Part D
a_1 = audio_equalizer(music_arr[:, 0], 0, 0 ,-40)
b_1 = audio_equalizer(music_arr[:, 1], 0, 0 ,-40)
combine_chan_1  = np.vstack([a_1/2 ,b_1/2]) 
chan_1 = combine_chan_1.T

wav.write('music_b', fs, chan_1.astype('int16'))
wav_music_b = sa.WaveObject.from_wave_file('music_b')
play_music_b = wav_music_b.play()
play_music_b.wait_done()

### Discussion

Larger gains lead to more obvious audible differences. A few restraints that we need to put on the gains include making sure that the gains affect frequencies within the human range of hearing as well as ensuring the gains do not saturate, ie. suprass or equal max gain of speaker system.